In [1]:
import pyscf
import pyscf.tools
import scipy
import numpy as np

from orbitalpartitioning import *

In [4]:
molecule = """
 Fe 1.67785607 0.00052233 0.06475932
 O 0.00000000 0.00000000 -0.47099074
 Fe -1.67785607 -0.00052233 0.06475932
 Cl 1.87002704 -1.09796437 1.99091682
 Cl 2.93244917 -0.98210488 -1.47467288
 Cl 2.37160936 2.07954091 -0.50446591
 Cl -1.87002704 1.09796437 1.99091682
 Cl -2.93244917 0.98210488 -1.47467288
 Cl -2.37160936 -2.07954091 -0.50446591
"""

basis = "def2-svp"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   True,
        spin    =   10, # number of unpaired electrons
        charge  =   -2,
        basis   =   basis)


pymol.build()
print("symmetry: ",pymol.topgroup)
# mf = pyscf.scf.UHF(pymol).x2c()
mf = pyscf.scf.ROHF(pymol).newton()
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "sad"

mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
# mf.analyze()

# Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()
ndocc = mf.nelec[1]
nsing = mf.nelec[0] - ndocc
nvirt = mf.mol.nao - ndocc - nsing

# Just use alpha orbitals
Cdocc = mf.mo_coeff[:,0:ndocc]
Csing = mf.mo_coeff[:,ndocc:ndocc+nsing]
Cvirt = mf.mo_coeff[:,ndocc+nsing:ndocc+nsing+nvirt]

nbas = Cdocc.shape[0]

symmetry:  C2


******** <class 'pyscf.scf.hf_symm.SymAdaptedROHF'> Newton solver flags ********
SCF tol = 1e-08
conv_tol_grad = 1e-05
max. SCF cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_cycle_inner = 12
max_stepsize = 0.05
ah_start_tol = 1e+09
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_start_cycle = 1
ah_max_cycle = 40
ah_grad_trust_region = 2.5
kf_interval = 4
kf_trust_region = 5
canonicalization = True
max_memory 4000 MB (current use 147 MB)
HOMO (B) = 0.318236980196595  LUMO (A) = 0.320979897364206
Initial guess E= -5347.0130535218  |g|= 4.92607
macro= 0  E= -5354.09525943033  delta_E= -7.08221  |g|= 1.2839  3 KF 15 JK
macro= 1  E= -5354.99700870197  delta_E= -0.901749  |g|= 0.380059  3 KF 16 JK
macro= 2  E= -5355.40185091219  delta_E= -0.404842  |g|= 0.324421  3 KF 17 JK
macro= 3  E= -5355.82121744053  delta_E= -0.419367  |g|= 0.146705  3 KF 18 JK
macro= 4  E= -5355.88710224331  delta_E= -0.0658848  |g|= 0.19974

# Define Fragments by AOs

In [6]:
# Find AO's corresponding to atoms
full = []
frag1 = []
frag2 = []
frag3 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 0:
        if ao[2] in ("3d", "4s"):
            frag1.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 1:
        if ao[2] in ("2p","3p"):
            frag2.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 2:
        if ao[2] in ("3d", "4s"):
            frag3.append(ao_idx)
            full.append(ao_idx)


frags = [frag1, frag2, frag3]
print(frags)


[[3, 14, 15, 16, 17, 18], [34, 35, 36, 37, 38, 39], [48, 59, 60, 61, 62, 63]]


# Define Projectors
We can choose to project onto the non-orthogonal AOs, or onto the symmetrically orthogonalized AOs.

In [7]:
# Define projectors
X = scipy.linalg.sqrtm(S)
X = np.eye(nbas) 
Pfull = X[:,full]  # non-orthogonal
Pf = []
for f in frags:
    Pf.append(X[:,f])


# Project MOs onto all fragments
For each orbital block (Docc, Sing, Virt), project each subspace onto the full list of fragment AOs. This will determine our full active space.

In [8]:
(Oact, Sact, Vact), (Cenv, Cerr, _) = svd_subspace_partitioning((Cdocc, Csing, Cvirt), Pfull, S)
assert(Cerr.shape[1] == 0)
Cact = np.hstack((Oact, Sact, Vact))

 Partition  184 orbitals into a total of   18 orbitals
            Index   Sing. Val. Space       
                0   1.50934548            2*
                1   1.41582271            2*
                2   1.40991659            2*
                3   1.13042337            0*
                4   0.97455806            1*
                5   0.97130876            1*
                6   0.97073165            1*
                7   0.96871902            1*
                8   0.96790716            1*
                9   0.95877868            1*
               10   0.95763740            1*
               11   0.95709205            1*
               12   0.94484108            1*
               13   0.94464798            1*
               14   0.82369819            0*
               15   0.81370206            0*
               16   0.77579159            0*
               17   0.77265666            0*
               18   0.66680021            2
               19   0.51975144            2
   

# Split active space into fragments

In [9]:
# Project active orbitals onto fragments
init_fspace = []
clusters = []
Cfrags = []
orb_index = 1

# import pickle

# data = {}
# data["Pf"] = Pf 
# data["Cdocc"] = Cdocc
# data["Csing"] = Csing
# data["Cvirt"] = Cvirt
# data["S"] = S
# data["frags"] = frags 

for fi,f in enumerate(frags):
    print()
    print(" Fragment: ", f)
    (Of, Sf, Vf), (_, _, _) = svd_subspace_partitioning((Oact, Sact, Vact), Pf[fi], S)
    Cfrags.append(np.hstack((Of, Sf, Vf)))
    ndocc_f = Of.shape[1]
    init_fspace.append((ndocc_f+Sf.shape[1], ndocc_f))
    nmof = Of.shape[1] + Sf.shape[1] + Vf.shape[1]
    clusters.append(list(range(orb_index, orb_index+nmof)))
    orb_index += nmof



# Orthogonalize Fragment orbitals
Cfrags = sym_ortho(Cfrags, S)

Cact = np.hstack(Cfrags)

# Write Molden files for visualization
pyscf.tools.molden.from_mo(mf.mol, "Pfull.molden", Pfull)
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
pyscf.tools.molden.from_mo(mf.mol, "Cenv.molden", Cenv)
for i in range(len(frags)):
    pyscf.tools.molden.from_mo(mf.mol, "Cfrag%i.molden"%i, Cfrags[i])

print(" init_fspace = ", init_fspace)
print(" clusters    = ", clusters)


# data["Cact"] = Cact
# data["init_fspace"] = init_fspace 
# data["clusters"] = clusters 
# with open('data_CrOCr.pickle', 'wb') as handle:
#     pickle.dump(data, handle)



 Fragment:  [3, 14, 15, 16, 17, 18]
 Partition   18 orbitals into a total of    6 orbitals
            Index   Sing. Val. Space       
                0   0.96411301            1*
                1   0.96131383            1*
                2   0.95024086            1*
                3   0.94032424            1*
                4   0.92632666            1*
                5   0.83009269            0*
                6   0.25082683            0
                7   0.16519458            0
                8   0.15599810            0
                9   0.04618792            2
               10   0.04433050            2
               11   0.03063804            2
               12   0.02633973            1
               13   0.00412583            0

 Fragment:  [34, 35, 36, 37, 38, 39]
 Partition   18 orbitals into a total of    6 orbitals
            Index   Sing. Val. Space       
                0   1.39744886            2*
                1   1.38808865            2*
               

# Make Integrals

In [10]:
print(Cenv.shape)
print(Cact.shape)
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

print(h.shape)

(184, 72)
(184, 18)
(184, 184)


In [11]:
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;

In [12]:
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)

In [13]:
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact @ Cact.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;


In [14]:
np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Cact)
np.save("overlap_mat", S)

Pa = mf.make_rdm1()[0]
Pb = mf.make_rdm1()[1]
np.save("Pa", Cact.T @ S @ Pa @ S @ Cact)
np.save("Pb", Cact.T @ S @ Pb @ S @ Cact)

In [15]:
import numpy as np
Ccmf = np.load("Ccmf_18.npy")
pyscf.tools.molden.from_mo(mf.mol, "Ccmf_18.molden", Ccmf)



WARN: orbitals [ 0  1  2  3  4  5 12 13 14 15 16 17] not symmetrized, norm = [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]

